In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)

# from folium import LayerControl
from eo_tools.util import show_insar_coh, show_insar_phi, show_sar_int

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [ ]:
# load a geometry
ids = [
 "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
 "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
]

In [ ]:
data_dir = "/data/S1"
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
out_dir = f"/data/res/Morocco-earthquake-2023"

In [ ]:
from eo_tools.S1.process import slc2geo, coherence, amplitude
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_coh = f"{out_dir}/coh.tif"
# computing amplitude and complex coherence  in the radar geometry
# coherence(file_prm, file_sec, file_coh, box_size=[3,10], magnitude=False)


In [ ]:
def coh_dask(file_prm, file_sec, file_out, box_size=5, magnitude=True):
    """Compute the complex coherence from two SLC image files.

    Args:
        file_prm (str): GeoTiff file of the primary SLC image
        file_sec (str): GeoTiff file of the secondary SLC image
        file_out (str): output file
        box_size (int, optional): Window size in pixels for boxcar filtering. Defaults to 5.
        magnitude (bool, optional): Writes magnitude only. Otherwise a complex valued raster is written. Defaults to True.
    """
    # from dask.distributed import Client, LocalCluster, Lock
    import dask.array as da
    import numpy as np

    # import rasterio as rio
    # import warnings
    # import rioxarray as riox
    import xarray as xr
    from eo_tools.S1.util import boxcar

    log.info("Computing coherence")

    if isinstance(box_size, list):
        box_az = box_size[0]
        box_rg = box_size[1]
    else:
        box_az = box_size
        box_rg = box_size

    def avg_ampl(arr, box_az, box_rg):
        return np.sqrt(boxcar((arr * arr.conj()).real, box_az, box_rg))

    prm = xr.open_dataset(file_prm, lock=False, chunks="auto", engine="rasterio")
    # da_prm = da.from_array(prm['band_data'][0], chunks=(1024,1024))
    args = {"dimaz": box_az, "dimrg": box_rg}
    # prm["band_data"][0].map_blocks(boxcar)#, template=prm['band_data'][0], kwargs=args)
    # prm["band_data"][0].map_blocks(boxcar, box_size[0], box_size[1], template=prm['band_data'][0])
    # prm["band_data"][0]
    # da.map_blocks(boxcar, prm["band_data"][0], box_az, box_rg, dtype=float, chunks=(500, 500))
    arr = da.map_blocks(
        boxcar,
        prm["band_data"][0].data,
        dimaz=box_az,
        dimrg=box_rg,
        dtype=float,
        chunks=(500, 500),
    )
    return abs(arr)

    # with rio.open(file_prm) as ds_prm:
    #     prm = da.from_array(ds_prm, chunks=(1000,1000))#.read(1)
    #     prof = ds_prm.profile
    # with rio.open(file_sec) as ds_sec:
    #     sec = da.from_array(ds_sec, chunks=(1000,1000))# .read(1)

    # coh = np.full_like(prm, np.nan + 1j * np.nan, dtype=prm.dtype)
    # log.info("coherence: cross-correlation")
    # coh = prm * sec.conj()
    # coh = boxcar(coh, box_az, box_rg)
    # with np.errstate(divide="ignore", invalid="ignore"):
    #     log.info("coherence: normalizing")
    #     coh /= avg_ampl(prm, box_az, box_rg)
    #     coh /= avg_ampl(sec, box_az, box_rg)

    # if magnitude:
    #     coh = np.abs(coh)

    # warnings.filterwarnings("ignore", category=rio.errors.NotGeoreferencedWarning)
    # prof.update(dict(dtype=coh.dtype.name))
    # with rio.open(file_out, "w", **prof) as dst:
    #     dst.write(coh, 1)
    # return prm


prm = coh_dask(file_prm, file_sec, file_coh, box_size=[3, 3], magnitude=False)

In [ ]:
# toto = prm.compute()
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.imshow(prm, interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 10))
# plt.imshow(abs(prm), interpolation="none")
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
prm['band_data'][0]

In [ ]:
from scipy.misc import face
from eo_tools.S1.util import boxcar
import dask.array as da
im = face().sum(2)

arr = da.from_array(im.astype(float))

# barr = arr.map_blocks(boxcar, 7, 15, dtype=float, chunks=(128,128))
barr = da.map_blocks(boxcar, arr, 7, 15, dtype=float, chunks=(128,128))
plt.figure(figsize=(10, 10))
plt.imshow(barr, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
da.map_blocks